В Apache Spark UDF (User-Defined Function) — это пользовательская функция, которая позволяет выполнять произвольные вычисления над данными в DataFrame. UDF можно использовать для применения сложной логики или вычислений, которые не поддерживаются стандартными функциями Spark SQL.

In [8]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession,DataFrame
from pyspark import StorageLevel
from pyspark.sql.functions import col, round, to_date, year, month, sum, avg, udf
from pyspark.sql.types import IntegerType, StringType

conf = (
     SparkConf()
    .setAppName('udf_Spark')
    .setMaster('local[*]')
)


# Spark сессия
spark = (
    SparkSession
    .builder
    .config(conf=conf)
    # .enableHiveSupport()
    .getOrCreate()
)

In [6]:
data = [(1, "Alice"), (2, "Bob"), (3, "Cathy"), (4, "David")]
df = spark.createDataFrame(data, ["id", "name"])

In [7]:
# Определяем пользовательскую функцию
def add_prefix(name):
    return f"Name_{name}"

In [9]:
# Регистрируем функцию как UDF
add_pref_udf = udf(add_prefix, StringType())

In [12]:
df.show()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
|  3|Cathy|
|  4|David|
+---+-----+



In [15]:
# применяем udf к df
prefixed_df = df.withColumn("prefixed_name", add_pref_udf(col("name")))
prefixed_df.show()b

+---+-----+-------------+
| id| name|prefixed_name|
+---+-----+-------------+
|  1|Alice|   Name_Alice|
|  2|  Bob|     Name_Bob|
|  3|Cathy|   Name_Cathy|
|  4|David|   Name_David|
+---+-----+-------------+



если Вы хотите использовать UDF в рамках чистого SQL, то конструкция создания UDF будет немного другая.

In [16]:
def name_length(name):
    return len(name)

# Регистрируем функцию как UDF с использованием spark.udf.register
spark.udf.register("name_length_udf", name_length, IntegerType())

# Создаем временную таблицу для выполнения SQL-запросов
df.createOrReplaceTempView("people")

# Используем зарегистрированную UDF в SQL-запросе
result_df = spark.sql("SELECT id, name, name_length_udf(name) as name_length FROM people")


result_df.show()

+---+-----+-----------+
| id| name|name_length|
+---+-----+-----------+
|  1|Alice|          5|
|  2|  Bob|          3|
|  3|Cathy|          5|
|  4|David|          5|
+---+-----+-----------+

